In [7]:
# Import pymongo module
from pymongo import MongoClient

# Create client, an instance and the collection
client = MongoClient('localhost', 27017)
db = client['mgo_db']
collection = db['time100_collection']

# Show how many documents in the collection
collection.count()

51565

<i><b>NOTE:</b> THE PREVIOUS BLOCK OF CELL CAN BE COMMENTED OUT WHEN IT IS NOT NECESSARY. IT NEED NOT TO BE RAN EACH TIME UNLESS YOU WANT TO GET THE LATEST TWEETS.</i>

In [8]:
# # Import pandas  module
# import pandas as pd

# # Load all the collection into 
# tweets = collection.find()

# # Import json_normalize and use this to normalize or flatten the JSON into a pandas dataframe
# from pandas.io.json import json_normalize
# df_tweets = json_normalize(tweets)

# # Leave out the noise
# df_tweets = df_tweets[['created_at', 'text', 'favorite_count', 'retweet_count', 'entities.hashtags','lang','user.screen_name',
#                        'user.name', 'user.followers_count', 'user.friends_count', 'user.time_zone', 'user.location', 
#                        'user.verified', 'user.profile_image_url', 'coordinates', 'coordinates.type', 'coordinates.coordinates',
#                        'place', 'place.place_type', 'place.name', 'place.full_name', 'place.country_code', 'place.country',
                       
#                        'retweeted_status.created_at', 'retweeted_status.text', 'retweeted_status.favorite_count', 
#                        'retweeted_status.retweet_count', 'retweeted_status.entities.hashtags', 'retweeted_status.lang',
#                        'retweeted_status.user.screen_name', 'retweeted_status.user.name', 'retweeted_status.user.followers_count', 
#                        'retweeted_status.user.friends_count', 'retweeted_status.user.time_zone', 'retweeted_status.user.location',
#                        'retweeted_status.coordinates', 'retweeted_status.coordinates.type', 'retweeted_status.coordinates.coordinates', 
#                        'retweeted_status.place', 'retweeted_status.place.place_type', 'retweeted_status.place.name', 
#                        'retweeted_status.place.full_name', 'retweeted_status.place.country_code', 'retweeted_status.place.country',
                        
#                        'quoted_status.created_at', 'quoted_status.text', 'quoted_status.favorite_count', 
#                        'quoted_status.retweet_count', 'quoted_status.entities.hashtags', 'quoted_status.lang',
#                        'quoted_status.user.screen_name', 'quoted_status.user.name', 'quoted_status.user.followers_count', 
#                        'quoted_status.user.friends_count', 'quoted_status.user.time_zone', 'quoted_status.user.location',
#                        'quoted_status.coordinates', 'quoted_status.coordinates.type', 'quoted_status.coordinates.coordinates', 
#                        'quoted_status.place', 'quoted_status.place.place_type', 'quoted_status.place.name', 
#                        'quoted_status.place.full_name', 'quoted_status.place.country_code', 'quoted_status.place.country'
#                       ]]

### SAVE A COPY IN CSV, SO THAT WE CAN COMMENT OUT AND SKIP THE PREVIOUS STEPS WHEN IT IS NOT NECESSARY. 
# df_tweets.to_csv('data/time100_tweets_0430_730PM.csv', index=False)

<i><b>NOTE:</b> THE PREVIOUS BLOCK OF CELL CAN BE COMMENTED OUT WHEN IT IS NOT NECESSARY. IT NEED NOT TO BE RAN EACH TIME UNLESS YOU RAN THE FIRST BLOCK.</i>

In [10]:
import pandas as pd
df_tweets = pd.read_csv('data/time100_tweets_0430_530PM.csv', encoding ='latin1', low_memory=False)
# Since we are only after the mentions of the Time 100 names in the tweets, select only the text 
# (or tweets or re-tweets or quated tweets. I might as well include all of these for larger data)
df_tweets = df_tweets[['text', 'retweeted_status.text', 'quoted_status.text']]
df_tweets.head()


,text,retweeted_status.text,quoted_status.text
0,RT @BillGates: Melindas work on behalf of wom...,Melindas work on behalf of women and girls in...,NaN
1,RT @TIME: Colin Kaepernicks willingness to t...,Colin Kaepernicks willingness to take a posi...,NaN
2,WWVDD? What Would Viola Davis Do?! Congratulat...,NaN,Meryl Streep on @ViolaDavis: Her gifts as an ...
3,RT @RitaPanahi: You're honoured to write about...,You're honoured to write about an anti-Semitic...,The four exceptional organizers of the @womens...
4,RT @TIME: See why Thailand's new king Maha Vaj...,See why Thailand's new king Maha Vajiralongkor...,NaN


In [13]:
# Import NLTK for processing tweets
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re

# We also parse emoticon as word, but since we are only after the names, this will just be ignored.
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

# As said, since we are only after the names, this will just be ignored.
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs 
    r'https://t.co/(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs 
    r'https://t.co/(?:[0-9]|[a-z]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

punctuation = list(string.punctuation)


# stop = stopwords.words('english')
stop = [] # Make is empty. Stopwords will also be ignore for now becasue we are only after the names

<i><b>NOTE:</b>  IN THE PREVIOUS BLOCK OF CELL, THERE ARE NLTK PREPROCOSSING VARIABLES THAT WILL BE IGNORED SINCE WE ARE ONLY MAKING A WORD CLOUD OF NAMES. 

ALSO, WE NEED NOT TO WORRY ABOUT STOPWORDS FOR NOW. UNCOMMENT THE ASSINGMENT OF stop WHEN YOU NEED TO GET THE ACTUAL WORDS IN THE TWEETS.</i>

In [14]:
# Import urlopen and BeautifulSoup modules
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Load the online page (where the list of names are listed, into an html object)
html = urlopen("http://time.com/collection/2017-time-100/")

# Using BeautifulSoup, parse the html
soup = BeautifulSoup(html, "html.parser")

# Local the tags where the names are listed and loop through each of them to get the names
tDivs = ('211','233', '252', '278', '294')
time100s = []
for tDiv in tDivs:
    aTags = soup.find('div', attrs={'data-reactid':tDiv}).find_all('a', href=True)
    for aTag in aTags:
        time100s.append(aTag.text)

# Split the names by separating the first and last, or any combincation of names.
inlist = []
for time100 in time100s:
    for name in time100.split():
        inlist.append(name)

In [15]:
time100s

['Samantha Bee',
 'Chance the Rapper',
 'Constance Wu',
 'Gavin Grimm',
 'Kirsten Green',
 'Bob Ferguson',
 'Ivanka Trump',
 'Demis Hassabis',
 'Barbara Lynch',
 'Hamdi Ulukaya',
 'Jared Kushner',
 'Celina Turchi',
 'Jordan Peele',
 'Glenda Gray',
 'Yuriko Koike',
 'Conor McGregor',
 'Riz Ahmed',
 'Guus Velders',
 'Tamika Mallory, Bob Bland, Carmen Perez and Linda Sarsour',
 'Natalie Batalha, Guillem Anglada-Escudé and Michaël Gillon',
 'Emma Stone',
 'Colson Whitehead',
 'Ed Sheeran',
 'Alicia Keys',
 'Ryan Reynolds',
 'Donald Glover',
 'Leslie Jones',
 'Ben Platt',
 'Ava DuVernay',
 'Barry Jenkins',
 'Margot Robbie',
 'Sarah Paulson',
 'James Corden',
 'John Legend',
 'Alessandro Michele',
 'Kerry James Marshall',
 'Demi Lovato',
 'Theresa May',
 'Narendra Modi',
 'Chuck Schumer',
 'Donald Trump',
 'Elizabeth Warren',
 'Julian Assange',
 'James Comey',
 'Kim Jong Un',
 'Reince Priebus',
 'Xi Jinping',
 'Rodrigo Duterte',
 'Stephen Bannon',
 'Theo Epstein',
 'Tom Perez',
 'Vladimir Pu

In [ ]:
# Import pandas and numpy modules
import numpy as np

# loop through each names
for time100 in time100s:
    
    # Save or open a text file with filename as their names. 
    # This file is where all of mentioned name/s in the tweets will be saved in individually.    
    with open('text/'+time100.replace(',', '').replace('.', '').replace("'", '').replace(' ', '_')+'.txt', 
              'w', encoding='latin1') as tt:
        
        # Now loop through the tweets in the dataframe
        for index, row in df_tweets.iterrows():
            for name in [name for name in time100.split() if name not in stop]:
                if re.findall('\\b'+name+'\\b', row['text'], flags=re.IGNORECASE):
                        new_tweet = ' '.join(str(e) for e in [term for term in preprocess(row['text']) 
                                                              # if term not in stop]) # Only the names we are after, not the stop words for now.
                                                              if term in inlist]) # Get only the names in the inlist
                        tt.write(new_tweet+" ")

                if row['retweeted_status.text'] != np.nan  \
                    and row['retweeted_status.text'] != ''  \
                    and not pd.isnull(row['retweeted_status.text']):
                        if re.findall('\\b'+name+'\\b', row['retweeted_status.text'], flags=re.IGNORECASE):
                            new_tweet = ' '.join(str(e) for e in [term for term in preprocess(row['retweeted_status.text']) 
                                                                  # if term not in stop]) # Only the names we are after, not the stop words for now.
                                                                  if term in inlist]) # Get only the names in the inlist
                            tt.write(new_tweet+" ")

                if row['quoted_status.text'] != np.nan \
                    and row['quoted_status.text'] != '' \
                    and not pd.isnull(row['quoted_status.text']) \
                    and 'https://t.co/' not in row['quoted_status.text']:
                        if re.findall('\\b'+name+'\\b', row['quoted_status.text'], flags=re.IGNORECASE):
                            new_tweet = ' '.join(str(e) for e in [term for term in preprocess(row['quoted_status.text']) 
                                                                  # if term not in stop]) # Only the names we are after, not the stop words for now.
                                                                  if term in inlist]) # Get only the names in the inlist
                            tt.write(new_tweet+" ")